In [3]:
from google.colab import files
import zipfile
import io
import os

# Step 1: Ask user to upload a ZIP file
print("Please upload your ZIP file:")
uploaded = files.upload()  # Opens file picker dialog

# Step 2: Extract the uploaded ZIP file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"Unzipping {filename}...")
        with zipfile.ZipFile(io.BytesIO(uploaded[filename]), 'r') as zip_ref:
            zip_ref.extractall('/content/')
        print("Extraction complete!")
        print("Files extracted to /content/")

# Step 3: List extracted files
print("\nExtracted folder structure:")
!ls -R /content/

Please upload your ZIP file:


Saving nas-ga-basic-main.zip to nas-ga-basic-main.zip
Unzipping nas-ga-basic-main.zip...
Extraction complete!
Files extracted to /content/

Extracted folder structure:
/content/:
nas-ga-basic-main  nas-ga-basic-main.zip  sample_data

/content/nas-ga-basic-main:
LICENSE  model_cnn.py  model_ga.py  nas_run.py	README.md

/content/sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [1]:
# ---------------------------
# Run both: Tournament vs Roulette
# ---------------------------
import os, shutil, zipfile, pickle, time
from pathlib import Path

ZIP_PATH = "/mnt/data/nas-ga-basic-main.zip"   # your uploaded zip (from conversation history)
CONTENT = "/content"
PROJECT_DIR = None
# === Logging helper ===
def enable_logging(run_id):
    run_dir = os.path.join(run_base, f'run_{run_id}')
    os.makedirs(run_dir, exist_ok=True)
    log_path = os.path.join(run_dir, "nas_run.log")

    log_file = open(log_path, "w")

    class Tee(object):
        def __init__(self, *files):
            self.files = files
        def write(self, data):
            for f in self.files:
                f.write(data)
        def flush(self):
            for f in self.files:
                try: f.flush()
                except: pass

    new_stdout = Tee(sys.stdout, log_file)
    new_stderr = Tee(sys.stderr, log_file)

    print(f"\n=== Logging enabled for run {run_id} ===")
    print(f"Output saved to: {log_path}\n")

    return log_file, new_stdout, new_stderr


# 0) Unzip if project folder not present
if not any(p for p in os.listdir(CONTENT) if 'nas' in p.lower() and os.path.isdir(os.path.join(CONTENT, p))):
    print("Project folder not found in /content. Unzipping uploaded ZIP...")
    if not os.path.exists(ZIP_PATH):
        raise FileNotFoundError(f"ZIP file not found at {ZIP_PATH}. Upload first or update ZIP_PATH.")
    shutil.copy(ZIP_PATH, CONTENT)
    zip_target = os.path.join(CONTENT, os.path.basename(ZIP_PATH))
    with zipfile.ZipFile(zip_target, 'r') as zf:
        zf.extractall(CONTENT)
    print("Unzipped to /content/")
else:
    print("Project folder found in /content/ - skipping unzip.")

# 1) Detect project folder (prefer one with 'nas' in name)
candidates = [p for p in os.listdir(CONTENT) if os.path.isdir(os.path.join(CONTENT, p))]
for c in candidates:
    if 'nas' in c.lower():
        PROJECT_DIR = os.path.join(CONTENT, c)
        break
if PROJECT_DIR is None:
    # fallback to most recently modified dir in /content
    dirs = [(os.path.join(CONTENT, d), os.path.getmtime(os.path.join(CONTENT, d))) for d in candidates]
    dirs.sort(key=lambda x: x[1], reverse=True)
    PROJECT_DIR = dirs[0][0] if dirs else CONTENT

print("Using project dir:", PROJECT_DIR)
# 2) Add project dir to sys.path and cd into it
import sys
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)
os.chdir(PROJECT_DIR)
print("Working directory:", os.getcwd())

# 3) Ensure outputs base exists
run_base = os.path.join(PROJECT_DIR, 'outputs')
os.makedirs(run_base, exist_ok=True)
# determine next two run ids
existing_runs = [d for d in os.listdir(run_base) if d.startswith('run_') and os.path.isdir(os.path.join(run_base, d))]
next_run = len(existing_runs) + 1
run_tournament = next_run
run_roulette = next_run + 1

# 4) Import project modules (model_ga, model_cnn). If import fails, raise informative error.
try:
    from model_ga import GeneticAlgorithm
    from model_cnn import CNN
except Exception as e:
    raise ImportError(f"Failed to import project modules (model_ga/model_cnn). Error: {e}")
# 5) Prepare dataset loaders (small subsets for quick runs)
import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
data_root = os.path.join(PROJECT_DIR, 'data')
os.makedirs(data_root, exist_ok=True)
trainset = CIFAR10(root=data_root, train=True, download=True, transform=transform)
valset = CIFAR10(root=data_root, train=False, download=True, transform=transform)
# small subsets to keep runtime short (adjust if you want longer runs)
train_subset = Subset(trainset, range(min(5000, len(trainset))))
val_subset   = Subset(valset,   range(min(1000, len(valset))))

train_loader = DataLoader(train_subset, batch_size=256, shuffle=True)
val_loader   = DataLoader(val_subset, batch_size=256, shuffle=False)
# helper logging
def log(msg):
    print(msg, flush=True)
# 6) Function to run an experiment (selection_method: 'tournament' or 'roulette')
def run_experiment(selection_method, population_size=5, generations=2, run_id=1):
    print("\n" + "="*60)
    print(f"STARTING RUN {run_id} (selection={selection_method})")
    print("="*60)

    ga = GeneticAlgorithm(
        population_size=population_size,
        generations=generations,
        mutation_rate=0.3,
        crossover_rate=0.7
    )

    t0 = time.time()
    best_arch = ga.evolve(train_loader, val_loader, device, run=run_id, selection_method=selection_method)
    elapsed = time.time() - t0

    run_dir = os.path.join(run_base, f'run_{run_id}')
    os.makedirs(run_dir, exist_ok=True)

    with open(os.path.join(run_dir, "best_arch.pkl"), 'wb') as f:
        pickle.dump(best_arch, f)

    print(f"Completed run_{run_id} in {elapsed:.1f} s. Best arch saved to {run_dir}/best_arch.pkl")

    summary = {
        'run_id': run_id,
        'selection': selection_method,
        'genes': best_arch.genes,
        'accuracy': best_arch.accuracy,
        'fitness_original': best_arch.fitness_original,
        'fitness_weighted': best_arch.fitness_weighted,
        'param_count': sum(p.numel() for p in CNN(best_arch.genes).parameters()),
        'run_dir': run_dir
    }
    return summary

# ---- RUN 1: TOURNAMENT ----
log_file1, new_out1, new_err1 = enable_logging(run_tournament)
orig_stdout, orig_stderr = sys.stdout, sys.stderr

sys.stdout, sys.stderr = new_out1, new_err1
summary_tourn = run_experiment("tournament", population_size=3, generations=5, run_id=run_tournament)
sys.stdout.flush(); sys.stderr.flush()
sys.stdout, sys.stderr = orig_stdout, orig_stderr
log_file1.close()


# ---- RUN 2: ROULETTE ----
log_file2, new_out2, new_err2 = enable_logging(run_roulette)
orig_stdout, orig_stderr = sys.stdout, sys.stderr

sys.stdout, sys.stderr = new_out2, new_err2
summary_roul = run_experiment("roulette", population_size=3, generations=5, run_id=run_roulette)
sys.stdout.flush(); sys.stderr.flush()
sys.stdout, sys.stderr = orig_stdout, orig_stderr
log_file2.close()


# ==========================
# 8) Final Comparison
# ==========================
print("\n" + "="*60)
print("COMPARISON SUMMARY")
print("="*60)

print(f"Run {summary_tourn['run_id']} (Tournament):")
print(f"  Accuracy: {summary_tourn['accuracy']}")
print(f"  Fitness (orig): {summary_tourn['fitness_original']}")
print(f"  Fitness (weighted): {summary_tourn['fitness_weighted']}")
print(f"  Params: {summary_tourn['param_count']}")
print(f"  Saved at: {summary_tourn['run_dir']}\n")

print(f"Run {summary_roul['run_id']} (Roulette):")
print(f"  Accuracy: {summary_roul['accuracy']}")
print(f"  Fitness (orig): {summary_roul['fitness_original']}")
print(f"  Fitness (weighted): {summary_roul['fitness_weighted']}")
print(f"  Params: {summary_roul['param_count']}")
print(f"  Saved at: {summary_roul['run_dir']}\n")

winner = summary_tourn if summary_tourn['fitness_original'] >= summary_roul['fitness_original'] else summary_roul

print("WINNER (by original fitness):")
print(f"  Run {winner['run_id']} - selection: {winner['selection']}")
print(f"  Accuracy: {winner['accuracy']}, Fitness (orig): {winner['fitness_original']}, Params: {winner['param_count']}")
# ==========================
# 8) Final Comparison
# ==========================
print("\n" + "="*60)
print("COMPARISON SUMMARY")
print("="*60)

print(f"Run {summary_tourn['run_id']} (Tournament):")
print(f"  Accuracy: {summary_tourn['accuracy']}")
print(f"  Fitness (orig): {summary_tourn['fitness_original']}")
print(f"  Fitness (weighted): {summary_tourn['fitness_weighted']}")
print(f"  Params: {summary_tourn['param_count']}")
print(f"  Saved at: {summary_tourn['run_dir']}\n")

print(f"Run {summary_roul['run_id']} (Roulette):")
print(f"  Accuracy: {summary_roul['accuracy']}")
print(f"  Fitness (orig): {summary_roul['fitness_original']}")
print(f"  Fitness (weighted): {summary_roul['fitness_weighted']}")
print(f"  Params: {summary_roul['param_count']}")
print(f"  Saved at: {summary_roul['run_dir']}\n")

winner = summary_tourn if summary_tourn['fitness_original'] >= summary_roul['fitness_original'] else summary_roul

print("WINNER (by original fitness):")
print(f"  Run {winner['run_id']} - selection: {winner['selection']}")
print(f"  Accuracy: {winner['accuracy']}, Fitness (orig): {winner['fitness_original']}, Params: {winner['param_count']}")
winner = (
    summary_tourn
    if (summary_tourn["fitness_weighted"] or 0) >= (summary_roul["fitness_weighted"] or 0)
    else summary_roul
)

print("WINNER (by WEIGHTED fitness):")
print(f"  Run {winner['run_id']}  |  Method: {winner['selection']}")
print(f"  Accuracy: {winner['accuracy']:.4f}")
print(f"  Fitness (weighted): {winner['fitness_weighted']:.6f}")
print(f"  Params: {winner['param_count']}")

winner = (
    summary_tourn
    if (summary_tourn['accuracy'] or 0) >= (summary_roul['accuracy'] or 0)
    else summary_roul
)

print("WINNER (by ACCURACY):")
print(f"  Run {winner['run_id']}  |  Method: {winner['selection']}")
print(f"  Accuracy: {winner['accuracy']:.4f}")
print(f"  Params: {winner['param_count']}")

print("\nAll done. Two runs completed and saved to outputs directory.")

Project folder found in /content/ - skipping unzip.
Using project dir: /content/nas-ga-basic-main
Working directory: /content/nas-ga-basic-main
Using device: cuda

=== Logging enabled for run 2 ===
Output saved to: /content/nas-ga-basic-main/outputs/run_2/nas_run.log


STARTING RUN 2 (selection=tournament)
Starting with 3 Population:
[Arch(conv=1, acc=0.0000, fit=0.0000), Arch(conv=4, acc=0.0000, fit=0.0000), Arch(conv=1, acc=0.0000, fit=0.0000)]


Generation 1/5
Evaluating architecture 1/3... Conv params: 9600, FC params: 8394250
Total params: 8403850
Accuracy: 0.541000
Original fitness: 0.456962
Weighted fitness: 0.540958
Fitness (orig): 0.4570, Accuracy: 0.5410
Weighted fitness: 0.5410
Evaluating architecture 2/3... Conv params: 213888, FC params: 1049994
Total params: 1263882
Accuracy: 0.589000
Original fitness: 0.576361
Weighted fitness: 0.588995
Fitness (orig): 0.5764, Accuracy: 0.5890
Weighted fitness: 0.5890
Evaluating architecture 3/3... Conv params: 960, FC params: 2099978
To

In [3]:
# ==========================
# 8) Final Comparison
# ==========================
print("\n" + "="*60)
print("COMPARISON SUMMARY")
print("="*60)

print(f"Run {summary_tourn['run_id']} (Tournament):")
print(f"  Accuracy: {summary_tourn['accuracy']}")
print(f"  Fitness (orig): {summary_tourn['fitness_original']}")
print(f"  Fitness (weighted): {summary_tourn['fitness_weighted']}")
print(f"  Params: {summary_tourn['param_count']}")
print(f"  Saved at: {summary_tourn['run_dir']}\n")

print(f"Run {summary_roul['run_id']} (Roulette):")
print(f"  Accuracy: {summary_roul['accuracy']}")
print(f"  Fitness (orig): {summary_roul['fitness_original']}")
print(f"  Fitness (weighted): {summary_roul['fitness_weighted']}")
print(f"  Params: {summary_roul['param_count']}")
print(f"  Saved at: {summary_roul['run_dir']}\n")

winner = summary_tourn if summary_tourn['fitness_original'] >= summary_roul['fitness_original'] else summary_roul

print("WINNER (by original fitness):")
print(f"  Run {winner['run_id']} - selection: {winner['selection']}")
print(f"  Accuracy: {winner['accuracy']}, Fitness (orig): {winner['fitness_original']}, Params: {winner['param_count']}")
winner = (
    summary_tourn
    if (summary_tourn["fitness_weighted"] or 0) >= (summary_roul["fitness_weighted"] or 0)
    else summary_roul
)

print("WINNER (by WEIGHTED fitness):")
print(f"  Run {winner['run_id']}  |  Method: {winner['selection']}")
print(f"  Accuracy: {winner['accuracy']:.4f}")
print(f"  Fitness (weighted): {winner['fitness_weighted']:.6f}")
print(f"  Params: {winner['param_count']}")

winner = (
    summary_tourn
    if (summary_tourn['accuracy'] or 0) >= (summary_roul['accuracy'] or 0)
    else summary_roul
)

print("WINNER (by ACCURACY):")
print(f"  Run {winner['run_id']}  |  Method: {winner['selection']}")
print(f"  Accuracy: {winner['accuracy']:.4f}")
print(f"  Params: {winner['param_count']}")
print("\nAll done. Two runs completed and saved to outputs directory.")
print("\nAll done. Two runs completed and saved to outputs directory.")


COMPARISON SUMMARY
Run 2 (Tournament):
  Accuracy: 0.646
  Fitness (orig): 0.63336118
  Fitness (weighted): 0.645994536142
  Params: 1263882
  Saved at: /content/nas-ga-basic-main/outputs/run_2

Run 3 (Roulette):
  Accuracy: 0.653
  Fitness (orig): 0.6085202200000001
  Fitness (weighted): 0.6529787522380001
  Params: 4447978
  Saved at: /content/nas-ga-basic-main/outputs/run_3

WINNER (by original fitness):
  Run 2 - selection: tournament
  Accuracy: 0.646, Fitness (orig): 0.63336118, Params: 1263882
WINNER (by WEIGHTED fitness):
  Run 3  |  Method: roulette
  Accuracy: 0.6530
  Fitness (weighted): 0.652979
  Params: 4447978
WINNER (by ACCURACY):
  Run 3  |  Method: roulette
  Accuracy: 0.6530
  Params: 4447978

All done. Two runs completed and saved to outputs directory.

All done. Two runs completed and saved to outputs directory.


In [ ]:
python

In [2]:
import shutil
from google.colab import files

# Path to your folder
folder_path = '/content/nas-ga-basic-main'

# Create ZIP file
zip_path = '/content/nas-ga-basic-main.zip'
shutil.make_archive('/content/nas-ga-basic-main', 'zip', folder_path)

# Download to local computer
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>